# Application Lesson 1: Application to Mitrani's hysteresis model
<img name="Mitrani_chain.jpg">

In [1]:
import marmote.core as marmotecore
import marmote.markovchain as marmotemarkovchain
import numpy as np

2025-02-10 15:09:16.890 INFO  [2261279] [marmote::log::initialize@51] ========= MARMOTE LOGGER IS INITIALIZED ==========


### Picture of the model (credit: Mitrani 2013, Figure 2)

<img src="./Mitrani_chain.png">

Defining the constants/parameters of the model

In [2]:
m = 4          # Number of reserve servers
n = 2          # Number of permanent servers
N = m+n        # Total number of servers
lambda_ = 3.0  # Arrival rate
mu_ = 1.0      # Individual server service rate
nu_ = 5.0      # Inverse of average warmup time of the block of m reserve servers
up = 6         # Upper threshold
down = 4       # Lower threshold
K = 10         # Queue truncation parameter

In [3]:
space = marmotecore.MarmoteBox( [ 3, K+1 ] )

In [4]:
space.Enumerate()

'(   0,   0) (   0,   1) (   0,   2) (   0,   3) (   0,   4) (   0,   5) (   0,   6) (   0,   7) (   0,   8) (   0,   9) (   0,  10) (   1,   0) (   1,   1) (   1,   2) (   1,   3) (   1,   4) (   1,   5) (   1,   6) (   1,   7) (   1,   8) (   1,   9) (   1,  10) (   2,   0) (   2,   1) (   2,   2) (   2,   3) (   2,   4) (   2,   5) (   2,   6) (   2,   7) (   2,   8) (   2,   9) (   2,  10) '

In [5]:
Qtrans = marmotecore.SparseMatrix( space )
Qtrans.set_type( marmotecore.CONTINUOUS )

In [6]:
Qtrans

marmotecore.SparseMatrix (SparseMatrix (Object at 0x55b78068fef0) :
- generator type: continuous
- number of origin states:      33
- number of destination states: 33
- number of non zero elements: 0
)

## Filling the matrix

### Naming elements to make code readable

In [7]:
# naming of coordinates
QUEUE = 1
SERV = 0
# naming of server states
SLOW = 0
WARMING = 1
FAST = 2

### Setting up the loop on states

In [8]:
stateBuffer = space.StateBuffer()
nextBuffer = space.StateBuffer()
space.FirstState(stateBuffer)
idx = 0
print(stateBuffer)

[0 0]


### Looping

In [9]:
looping = True
while looping:
    # print( "Transitions for state ", stateBuffer )
    # convenience local variables, also used for restoring stateBuffer
    q = stateBuffer[QUEUE]
    s = stateBuffer[SERV]
    
    nextBuffer = np.array(stateBuffer) # copy current state
    # Event: arrivals
    if ( q < K ):
      nextBuffer[QUEUE] += 1;
      if ( ( nextBuffer[QUEUE] > up ) and ( nextBuffer[SERV] == SLOW ) ):
        nextBuffer[SERV] = WARMING;
      
      Qtrans.addToEntry( idx, space.Index(nextBuffer), lambda_ );
      Qtrans.addToEntry( idx, idx, -lambda_ );
    
    nextBuffer = np.array(stateBuffer) # copy current state
    # Event: departure
    if ( q > 0 ):
        # number of active servers
        if ( s == FAST ):
            nbServ = min( q, n+m )
        else:
            nbServ = min( q, n )
        nextBuffer[QUEUE] -= 1
        if ( nextBuffer[QUEUE] == down ): # whatever state of server: becomes SLOW
            nextBuffer[SERV] = SLOW;
      
        Qtrans.addToEntry( idx, space.Index(nextBuffer), mu_ * nbServ )
        Qtrans.addToEntry( idx, idx, -mu_ * nbServ )
    
    nextBuffer = np.array(stateBuffer) # copy current state
    # Event: end of warmup
    if ( s == WARMING ):
        nextBuffer[SERV] = FAST
        Qtrans.addToEntry( idx, space.Index(nextBuffer), nu_ )
        Qtrans.addToEntry( idx, idx, -nu_ )
    
    # next state
    space.NextState( stateBuffer )
    idx += 1
    looping = not space.IsFirst(stateBuffer)

### Inspecting the matrix. 

Matrices can be printed using a variety of formats (sparse, Matlab, Maple, numpy).

In [10]:
print( Qtrans.toString( marmotecore.FORMAT_NUMPY ) )

mc_matrix=np.array([
[-3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[1, -4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 2, -5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 2, -5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 2, -5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 2, -5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 2, -5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 2, -5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 2, -5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 2, -5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

The `FullDiagnose` method produces a summary of various metrics **plus** the structural analysis of the matrix (recurrent/transient classes).

In [11]:
marmotecore.setStateWritePolicy( marmotecore.STATE_INDEX )
Qtrans.FullDiagnose()

# Generator general diagnostic:
Diagnostic for SparseMatrix structure:
- generator type:        continuous
- number of origin states:      33
- number of destination states: 33
- number of transitions: 104
- number of empty rows:  0
- maximum outdegree:     4
- minimum outdegree:     2
- maximum indegree:      4
- minimum indegree:      0
- maximum value:                    6
- minimum value:                  -10
- maximum row sum:                  0
- minimum row sum:                  0
- row sum mismatch:                 0
# Communicating classes:
number = 7
list = ( [ 0 1 2 3 4 5 6 16 17 18 19 20 21 27 28 29 30 31 32 ] [ 7 ] [ 8 ] [ 9 ] [ 10 ] [ 11 12 13 14 15 ] [ 22 23 24 25 26 ] )
# connectivity:
0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 
1.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 
1.00000000 1.00000000 0.00000000 0.00000000 0.00000000 0.00000000 0.00000000 
1.00000000 0.00000000 1.00000000 0.00000000 0.00000000 

Since it is not always simple to relate indices to states, it is also possible to print both the index **and** the state.

In [12]:
marmotecore.setStateWritePolicy( marmotecore.STATE_BOTH )
Qtrans.FullDiagnose()

# Generator general diagnostic:
Diagnostic for SparseMatrix structure:
- generator type:        continuous
- number of origin states:      33
- number of destination states: 33
- number of transitions: 104
- number of empty rows:  0
- maximum outdegree:     4
- minimum outdegree:     2
- maximum indegree:      4
- minimum indegree:      0
- maximum value:                    6
- minimum value:                  -10
- maximum row sum:                  0
- minimum row sum:                  0
- row sum mismatch:                 0
# Communicating classes:
number = 7
list = ( [ 0:{   0   0} 1:{   0   1} 2:{   0   2} 3:{   0   3} 4:{   0   4} 5:{   0   5} 6:{   0   6} 16:{   1   5} 17:{   1   6} 18:{   1   7} 19:{   1   8} 20:{   1   9} 21:{   1  10} 27:{   2   5} 28:{   2   6} 29:{   2   7} 30:{   2   8} 31:{   2   9} 32:{   2  10} ] [ 7:{   0   7} ] [ 8:{   0   8} ] [ 9:{   0   9} ] [ 10:{   0  10} ] [ 11:{   1   0} 12:{   1   1} 13:{   1   2} 14:{   1   3} 15:{   1   4} ] [ 22:{   2   0} 23

In [13]:
hymc = marmotemarkovchain.MarkovChain( Qtrans )
hymc.set_model_name( "Hysteresis_box" )

In [14]:
# TODO something wrong creates segfault in this line.# 
print( [ hymc.IsAccessible(3,31), hymc.IsAccessible(3,6), hymc.IsAccessible(3,10), hymc.IsAccessible(1,30), hymc.IsAccessible(30,27) ] )
print( hymc.IsIrreducible() )

[True, True, False, True, True]
False


## Stationary distribution and average cost

Let us compute the stationary distribution.

In [15]:
dista = hymc.StationaryDistribution()

Testing equality between Box( [ 0..2 ] x [ 0..10 ] ) and Box( [ 0..2 ] x [ 0..10 ] ): is 1 and true is 1


As expected, the distribution has many zeroes.

In [16]:
print(dista)

Discrete distribution values { 0:{   0   0}  1:{   0   1}  2:{   0   2}  3:{   0   3}  4:{   0   4}  5:{   0   5}  6:{   0   6}  7:{   0   7}  8:{   0   8}  9:{   0   9}  10:{   0  10}  11:{   1   0}  12:{   1   1}  13:{   1   2}  14:{   1   3}  15:{   1   4}  16:{   1   5}  17:{   1   6}  18:{   1   7}  19:{   1   8}  20:{   1   9}  21:{   1  10}  22:{   2   0}  23:{   2   1}  24:{   2   2}  25:{   2   3}  26:{   2   4}  27:{   2   5}  28:{   2   6}  29:{   2   7}  30:{   2   8}  31:{   2   9}  32:{   2  10}  } probas {  0.01773  0.05318  0.07977   0.1197   0.1795   0.1417  0.08502        0        0        0        0        0        0        0        0        0 0.001245 0.006223  0.02925 0.009391 0.003082 0.001321        0        0        0        0        0  0.05051  0.06631  0.06903  0.04601  0.02667  0.01444 }


Computing an average linear cost.

In [17]:
def avg_cost( c1, c2, dis ):
    avgL = 0
    avgS = 0
    space.FirstState(stateBuffer)
    # print(stateBuffer)
    go_on = True
    index = 0
    while go_on:
        prob = dis.getProbaByIndex(index)
        nbServ = n
        if ( stateBuffer[SERV] == FAST ):
            nbServ += m
        avgL = avgL + prob*stateBuffer[QUEUE]
        avgS = avgS + prob*nbServ
        space.NextState(stateBuffer)
        # print(stateBuffer)
        index = index+1
        go_on = not space.IsFirst(stateBuffer)
    return( c1*avgL + c2*avgS )

In [18]:
cost = avg_cost( 1.0, 1.0, dista )
print(cost)

7.850531279190159


And checking that the distribution is consistent.

In [19]:
total = 0
for i in range(space.Cardinal()):
    total = total + dista.getProbaByIndex(i)
print(total)

0.9999999999999972


## Tayloring the state space

Now defining a state space which fits exactly the recurrent class

In [20]:
smaller_space = marmotecore.MarmoteUnionSet()
smaller_space.AddSet( marmotecore.MarmoteInterval(0,up) )
smaller_space.AddSet( marmotecore.MarmoteInterval(down+1,K) )
smaller_space.AddSet( marmotecore.MarmoteInterval(down+1,K) )

In [21]:
smaller_space.Enumerate()

' 0:[ 0]  0:[ 1]  0:[ 2]  0:[ 3]  0:[ 4]  0:[ 5]  0:[ 6]  1:[ 5]  1:[ 6]  1:[ 7]  1:[ 8]  1:[ 9]  1:[ 10]  2:[ 5]  2:[ 6]  2:[ 7]  2:[ 8]  2:[ 9]  2:[ 10] '

In [22]:
smaller_space.Cardinal()

19

In [23]:
smaller_space.Belongs( [ 2, 3 ] )

False

Defining a new matrix on this smaller state space, and filling it.

In [24]:
Qtrans_alt = marmotecore.SparseMatrix( smaller_space )
Qtrans_alt.set_type( marmotecore.CONTINUOUS )

In [25]:
# naming of coordinates
QUEUE = 1
SERV = 0
# naming of server states
SLOW = 0
WARMING = 1
FAST = 2
# preparation of buffers
stateBuffer = smaller_space.StateBuffer()
smaller_space.FirstState(stateBuffer)
looping = True
idx = 0
while looping:
    # print( "Transitions for state ", stateBuffer )
    # convenience local variables, also used for restoring stateBuffer
    q = stateBuffer[QUEUE]
    s = stateBuffer[SERV]

    nextBuffer = np.array(stateBuffer) # copy current state
    # Event: arrivals
    if ( q < K ):
      nextBuffer[QUEUE] += 1;
      if ( ( nextBuffer[QUEUE] > up ) and ( nextBuffer[SERV] == SLOW ) ):
        nextBuffer[SERV] = WARMING;
      
      Qtrans_alt.addToEntry( idx, smaller_space.Index(nextBuffer), lambda_ );
      Qtrans_alt.addToEntry( idx, idx, -lambda_ );
      # print( stateBuffer, " to ", nextBuffer )
      # print( smaller_space.Belongs(nextBuffer), smaller_space.Index(nextBuffer) )
    
    nextBuffer = np.array(stateBuffer) # copy current state
    # Event: departure
    if ( q > 0 ):
        # number of active servers
        if ( s == FAST ):
            nbServ = min( q, n+m )
        else:
            nbServ = min( q, n )
        nextBuffer[QUEUE] -= 1
        if ( nextBuffer[QUEUE] == down ): # whatever state of server: becomes SLOW
            nextBuffer[SERV] = SLOW;
      
        Qtrans_alt.addToEntry( idx, smaller_space.Index(nextBuffer), mu_ * nbServ )
        Qtrans_alt.addToEntry( idx, idx, -mu_ * nbServ )
    
    nextBuffer = np.array(stateBuffer) # copy current state
    # Event: end of warmup
    if ( s == WARMING ):
        nextBuffer[SERV] = FAST
        Qtrans_alt.addToEntry( idx, smaller_space.Index(nextBuffer), nu_ )
        Qtrans_alt.addToEntry( idx, idx, -nu_ )
    
    # next state
    smaller_space.NextState( stateBuffer )
    idx += 1
    looping = not smaller_space.IsFirst(stateBuffer)

In [26]:
print( Qtrans_alt.toString( marmotecore.FORMAT_NUMPY ) )

mc_matrix=np.array([
[-3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[1, -4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 2, -5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 2, -5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 2, -5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 2, -5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 2, -5, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 2, 0, 0, -10, 3, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 2, -10, 3, 0, 0, 0, 0, 5, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 2, -10, 3, 0, 0, 0, 0, 5, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, -10, 3, 0, 0, 0, 0, 5, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, -10, 3, 0, 0, 0, 0, 5, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, -7, 0, 0, 0, 0, 0, 5],
[0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, -8, 3, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, -9, 3, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, -9, 3, 0, 0],
[0, 0, 0, 0, 0

Using the "full state" feature to inspect in detail the transitions.

In [27]:
print( Qtrans_alt.toString( marmotecore.FORMAT_FULLSTATE ) )

 0:{ 0 0}   1:{ 0 1}  3.000000e+00
 0:{ 0 0}   0:{ 0 0}  -3.000000e+00
 1:{ 0 1}   2:{ 0 2}  3.000000e+00
 1:{ 0 1}   1:{ 0 1}  -4.000000e+00
 1:{ 0 1}   0:{ 0 0}  1.000000e+00
 2:{ 0 2}   3:{ 0 3}  3.000000e+00
 2:{ 0 2}   2:{ 0 2}  -5.000000e+00
 2:{ 0 2}   1:{ 0 1}  2.000000e+00
 3:{ 0 3}   4:{ 0 4}  3.000000e+00
 3:{ 0 3}   3:{ 0 3}  -5.000000e+00
 3:{ 0 3}   2:{ 0 2}  2.000000e+00
 4:{ 0 4}   5:{ 0 5}  3.000000e+00
 4:{ 0 4}   4:{ 0 4}  -5.000000e+00
 4:{ 0 4}   3:{ 0 3}  2.000000e+00
 5:{ 0 5}   6:{ 0 6}  3.000000e+00
 5:{ 0 5}   5:{ 0 5}  -5.000000e+00
 5:{ 0 5}   4:{ 0 4}  2.000000e+00
 6:{ 0 6}   9:{ 1 7}  3.000000e+00
 6:{ 0 6}   6:{ 0 6}  -5.000000e+00
 6:{ 0 6}   5:{ 0 5}  2.000000e+00
 7:{ 1 5}   8:{ 1 6}  3.000000e+00
 7:{ 1 5}   7:{ 1 5}  -1.000000e+01
 7:{ 1 5}   4:{ 0 4}  2.000000e+00
 7:{ 1 5}  13:{ 2 5}  5.000000e+00
 8:{ 1 6}   9:{ 1 7}  3.000000e+00
 8:{ 1 6}   8:{ 1 6}  -1.000000e+01
 8:{ 1 6}   7:{ 1 5}  2.000000e+00
 8:{ 1 6}  14:{ 2 6}  5.000000e+00
 9:{ 1 7}  

Looking at the structural analysis.

Now all states are recurrent.

In [28]:
Qtrans_alt.FullDiagnose()

# Generator general diagnostic:
Diagnostic for SparseMatrix structure:
- generator type:        continuous
- number of origin states:      19
- number of destination states: 19
- number of transitions: 60
- number of empty rows:  0
- maximum outdegree:     4
- minimum outdegree:     2
- maximum indegree:      4
- minimum indegree:      1
- maximum value:                    6
- minimum value:                  -10
- maximum row sum:                  0
- minimum row sum:                  0
- row sum mismatch:                 0
# Communicating classes:
number = 1
list = ( [ 0:{ 0 0} 1:{ 0 1} 2:{ 0 2} 3:{ 0 3} 4:{ 0 4} 5:{ 0 5} 6:{ 0 6} 7:{ 1 5} 8:{ 1 6} 9:{ 1 7} 10:{ 1 8} 11:{ 1 9} 12:{ 1 10} 13:{ 2 5} 14:{ 2 6} 15:{ 2 7} 16:{ 2 8} 17:{ 2 9} 18:{ 2 10} ] )
# connectivity:
0.00000000 
# Recurrent classes:
number = 1
list = ( [ 0:{ 0 0} 1:{ 0 1} 2:{ 0 2} 3:{ 0 3} 4:{ 0 4} 5:{ 0 5} 6:{ 0 6} 7:{ 1 5} 8:{ 1 6} 9:{ 1 7} 10:{ 1 8} 11:{ 1 9} 12:{ 1 10} 13:{ 2 5} 14:{ 2 6} 15:{ 2 7} 16:{ 2 8} 17:{ 

Indeed, irreducibility can be tested on Markov chains with the `IsIrreducible()` method.

In [29]:
hymc_alt = marmotemarkovchain.MarkovChain( Qtrans_alt )
print( hymc_alt.IsIrreducible() )
print( hymc.IsIrreducible() )

True
False


Computing the stationary distribution of this new chain, then computing the average cost.

In [30]:
dista_alt = hymc_alt.StationaryDistribution()

Testing equality between some 15MarmoteUnionSet and some 15MarmoteUnionSet
Testing equality between some 15MarmoteUnionSet and some 15MarmoteUnionSet


Testing equality between  0 0
 0 1
 0 2
 0 3
 0 4
 0 5
 0 6
 1 5
 1 6
 1 7
 1 8
 1 9
 1 10
 2 5
 2 6
 2 7
 2 8
 2 9
 2 10
 and  0 0
 0 1
 0 2
 0 3
 0 4
 0 5
 0 6
 1 5
 1 6
 1 7
 1 8
 1 9
 1 10
 2 5
 2 6
 2 7
 2 8
 2 9
 2 10
: is 1 and true is 1


In [31]:
print(dista_alt)
total = 0
for i in range(smaller_space.Cardinal()):
    total = total + dista_alt.getProbaByIndex(i)
print(total)

Discrete distribution values { 0:{ 0 0}  1:{ 0 1}  2:{ 0 2}  3:{ 0 3}  4:{ 0 4}  5:{ 0 5}  6:{ 0 6}  7:{ 1 5}  8:{ 1 6}  9:{ 1 7}  10:{ 1 8}  11:{ 1 9}  12:{ 1 10}  13:{ 2 5}  14:{ 2 6}  15:{ 2 7}  16:{ 2 8}  17:{ 2 9}  18:{ 2 10}  } probas {  0.01773  0.05318  0.07977   0.1197   0.1795   0.1417  0.08502 0.001245 0.006223  0.02925 0.009391 0.003082 0.001321  0.05051  0.06631  0.06903  0.04601  0.02667  0.01444 }
0.9999999999999972


In [32]:
def avg_cost_alt( c1, c2, dis ):
    avgL = 0
    avgS = 0
    smaller_space.FirstState(stateBuffer)
    # print(stateBuffer)
    go_on = True
    index = 0
    while go_on:
        prob = dis.getProbaByIndex(index)
        nbServ = n
        if ( stateBuffer[SERV] == FAST ):
            nbServ += m
        avgL = avgL + prob*stateBuffer[QUEUE]
        avgS = avgS + prob*nbServ
        smaller_space.NextState(stateBuffer)
        # print(stateBuffer)
        index = index+1
        go_on = not smaller_space.IsFirst(stateBuffer)
    return( c1*avgL + c2*avgS )

In [33]:
cost_alt = avg_cost_alt( 1.0, 1.0, dista_alt )

In the end we have the same cost

In [34]:
print(cost_alt)
print( cost )

7.850531279190159
7.850531279190159
